# Get started with Azure Cosmos DB development

Built-in Jupyter notebooks in Azure Cosmos DB allow to easily ingest and view your data - all within the Azure Portal. In this notebook, we'll learn how to create an Azure Cosmos container, ingest data, and query data.

To run this C# notebook, be sure to select the **CSharp** kernel in the command bar, so we can get all the language support features we need.


## Create database and containers
First, we'll create a new database and container to hold our data. Note when we create the container, we select a value for the partition key: we'll partition our data by the item id value, as it has a high cardinality (important for workloads during a lot of writes, e.g. IOT workloads) and evenly distributes the request and storage volume. Choosing a good partition key is "key" to getting good scale and performance from Azure Cosmos DB, so it's important we follow the [best practices](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview)!

### The type of container this will create depends on the capacity mode of your Cosmos DB account:
- On a provisioned throughput account, this will create a container provisioned at 400 RU/s, the minimum RU/s for any provisioned throughput container.
- On a serverless account, this will create a serverless container.

In [2]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("WebsiteData");
Container container = await database.CreateContainerIfNotExistsAsync("Sales", "/id");

Display.AsMarkdown(@"
Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.
");


Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.


In [17]:
%%upload --databaseName WebsiteData --containerName Sales --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData-small.json

Documents successfully uploaded to Sales 
Total number of documents imported: 
  Success: 7 
  Failure: 0 
Total time taken : 00:00:00 hours 
Total RUs consumed : 74.69

## Querying your data

Next, we'll write some SQL queries to explore the data in the Sales container. In Azure Cosmos DB SQL API, you can query your data with SQL queries. Azure Cosmos DB SQL is different from the ANSI SQL you might already be familiar with because it's designed for querying JSON documents, as opposed to relational database tables. However, most SQL concepts still apply in Azure Cosmos DB and if you're already familar with SQL, it is easy to learn how to write queries in Azure Cosmos DB.

### Let's explore some examples below.

In [1]:
%%sql --database WebsiteData --container Sales
SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 17.58 queryCompileTimeInMs : 0.48 queryLogicalPlanBuildTimeInMs : 0.06 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 16.479999999999997 indexLookupTimeInMs : 0 documentLoadTimeInMs : 10.719999999999999 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2654 retrievedDocumentSize : 1365585 outputDocumentCount : 2654 outputDocumentSize : 220967 writeOutputTimeInMs : 2.09 averageIndexUtilizationRation : 1

,Action,ItemRevenue,Country,Item
0,Viewed,350.00,Iceland,Cosmos T-shirt
1,Viewed,80.00,Paraguay,Light Jeans
2,Purchased,20.00,Central African Republic,Gloves
3,Added,31.99,Tuvalu,Denim Jacket
4,Viewed,79.99,Christmas Island,Walking Shoes
...,...,...,...,...
2649,Viewed,90.00,Honduras,Dark Jeans
2650,Added,22.50,Tonga,Athletic Shorts
2651,Added,90.00,Sao Tome and Principe,Dark Jeans
2652,Viewed,80.00,Algeria,Light Jeans
